In [32]:
import pandas as pd
import re

import past_guest_module as pm

import sys
sys.path.append('../zoho_data_app/')

import main_module as m

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
path = r'C:\Users\fajar.fatoni\Documents\Python\Data\check_country_code_analytic.xlsx'
df_analytic = pd.read_excel(path, dtype='string', sheet_name='Sheet1')
count_data_awal = df_analytic.shape[0]
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB047991,Karma Resorts,Other,Past Guests,Karma Bavaria,PF305258,Mr,David,Pycock,Male,...,2025-02-27 00:00:00,4,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
1,KK067437,Karma Resorts,Other,Past Guests,Karma Kandara,PF298909,Mr,N,S,Male,...,2025-03-02 00:00:00,1,DIGITAL,GHPA,DIGITAL,2,0,0,Email and Phone,NO
2,KK067616,Karma Resorts,Other,Past Guests,Karma Kandara,PF261706,Ms,Ros,Smith,Female,...,2025-02-10 00:00:00,4,HOUSE_USE,V_OWNER_NRB,COMP,3,0,0,Email and Phone,NO
3,KK070336,Karma Resorts,Other,Past Guests,Karma Kandara,PF304814,Mr,.,Nir,Male,...,2025-02-23 00:00:00,2,HOUSE_USE,V_OWNER_NRB,COMP,4,0,0,Email and Phone,NO
4,KB047602,Karma Resorts,Other,Past Guests,Karma Bavaria,PF301905,Mr,Justin,Wells,Male,...,2025-02-24 00:00:00,3,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,KB046605,Karma Resorts,Other,Past Guests,Karma Bavaria,<NA>,Firma,Karma,Bavaria,Unknown,...,2025-02-12 10:30:00,0,LGR,LGR,GROUP,1,0,0,Email Only,NO
265,KB039565,Karma Resorts,Other,Past Guests,Karma Bavaria,<NA>,<NA>,Bayri. Textislverband,-,Unknown,...,2025-02-23 18:00:00,2,CB,CB,GROUP,1,0,0,Email Only,NO
266,KB047084,Karma Resorts,Other,Past Guests,Karma Bavaria,<NA>,Firma,Karma,Bavaria,Unknown,...,2025-02-28 23:00:00,50,LGR,LGR,GROUP,1,0,0,Email Only,NO
267,KB039563,Karma Resorts,Other,Past Guests,Karma Bavaria,<NA>,<NA>,Bayri. Textislverband,-,Unknown,...,2025-02-09 18:00:00,2,CB,CB,GROUP,1,0,0,Email Only,NO


In [34]:
df_analytic.columns

Index(['Booking_Id', 'Lead Brand', 'Lead Sub- Brand', 'Lead Source',
       'Resort_Name', 'ProfileRef', 'Salutation', 'First_Name', 'Surname',
       'Gender', 'Language', 'DOB', 'Street', 'Area', 'Town', 'County',
       'Country', 'Nationality', 'PostCode', 'TelNo', 'MobileNo', 'Email',
       'Exclude From Mailing', 'Opt In', 'Booking Status', 'DateArrival',
       'DateDepart', 'No. of Nights', 'Market_Segment', 'Media_Code',
       'Media Code Group', 'No. of Adult', 'No. of Children', 'No. of Infants',
       'Contact Type', 'Odyssey Members'],
      dtype='object')

In [35]:
df_analytic.fillna('empty', inplace=True)

In [36]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


In [37]:
m.clean_space(df_email, 'Email')
df_email['Email'] = m.lowercase(df_email, 'Email')

c:\Users\fajar.fatoni\Documents\Python\Task Offline\past_guest_toni_app\../zoho_data_app\main_module.py:94: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)


In [38]:
# hitung jumlah duplicate
count_duplicate_general = df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])].shape[0]

df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])]

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
30,KB045686,Karma Resorts,Other,Past Guests,Karma Bavaria,PF293005,Mr,Fabrice,Sick,Male,...,2025-03-01 00:00:00,103,COMP,STAFF,COMP,1,0,0,Email Only,NO
34,KB048312,Karma Resorts,Other,Past Guests,Karma Bavaria,PF305150,Herr,Andreii,Coi,Male,...,2025-02-26 00:00:00,2,COMP,STAFF,COMP,1,0,0,Email Only,NO
40,KB045542,Karma Resorts,Other,Past Guests,Karma Bavaria,PF209139,Frau,Christina,Just,Female,...,2025-02-12 18:00:00,0,CB,EMAIL_MRKTNG,DIGITAL,1,0,0,Email and Phone,NO
42,KB048507,Karma Resorts,Other,Past Guests,Karma Bavaria,PF080008,empty,Kea,Marketing Gmbh,Unknown,...,2025-03-02 21:00:00,0,LGR,LGR,GROUP,1,0,0,Email Only,NO
43,KB046080,Karma Resorts,Other,Past Guests,Karma Bavaria,PF294712,Firma,Siedlungswasserwirtschaft-Tum,.,Unknown,...,2025-02-28 18:00:00,0,CB,OTA,RETAIL,1,0,0,Email Only,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,KB046605,Karma Resorts,Other,Past Guests,Karma Bavaria,empty,Firma,Karma,Bavaria,Unknown,...,2025-02-12 10:30:00,0,LGR,LGR,GROUP,1,0,0,Email Only,NO
265,KB039565,Karma Resorts,Other,Past Guests,Karma Bavaria,empty,empty,Bayri. Textislverband,-,Unknown,...,2025-02-23 18:00:00,2,CB,CB,GROUP,1,0,0,Email Only,NO
266,KB047084,Karma Resorts,Other,Past Guests,Karma Bavaria,empty,Firma,Karma,Bavaria,Unknown,...,2025-02-28 23:00:00,50,LGR,LGR,GROUP,1,0,0,Email Only,NO
267,KB039563,Karma Resorts,Other,Past Guests,Karma Bavaria,empty,empty,Bayri. Textislverband,-,Unknown,...,2025-02-09 18:00:00,2,CB,CB,GROUP,1,0,0,Email Only,NO


In [39]:
df_analytic.drop_duplicates(subset=['TelNo','MobileNo','Email'], inplace=True)
df_analytic.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = df_analytic.shape[0]

df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB047991,Karma Resorts,Other,Past Guests,Karma Bavaria,PF305258,Mr,David,Pycock,Male,...,2025-02-27 00:00:00,4,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
1,KK067437,Karma Resorts,Other,Past Guests,Karma Kandara,PF298909,Mr,N,S,Male,...,2025-03-02 00:00:00,1,DIGITAL,GHPA,DIGITAL,2,0,0,Email and Phone,NO
2,KK067616,Karma Resorts,Other,Past Guests,Karma Kandara,PF261706,Ms,Ros,Smith,Female,...,2025-02-10 00:00:00,4,HOUSE_USE,V_OWNER_NRB,COMP,3,0,0,Email and Phone,NO
3,KK070336,Karma Resorts,Other,Past Guests,Karma Kandara,PF304814,Mr,.,Nir,Male,...,2025-02-23 00:00:00,2,HOUSE_USE,V_OWNER_NRB,COMP,4,0,0,Email and Phone,NO
4,KB047602,Karma Resorts,Other,Past Guests,Karma Bavaria,PF301905,Mr,Justin,Wells,Male,...,2025-02-24 00:00:00,3,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,KLM008147,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF301504,Ms,Marilyn,Muir,Female,...,2025-02-15 00:00:00,1,DIR,WEBSITE,DIGITAL,2,0,0,Email and Phone,NO
179,KSH022320,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF299978,empty,Helen,Grant,Unknown,...,2025-02-25 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
180,KSH023054,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF303825,Ms,Emma,Mcloughlin,Female,...,2025-02-14 00:00:00,1,COMP,WEDDING,GROUP,2,0,0,Email Only,NO
181,KLM007347,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF293361,empty,Stuart,Fleming,Unknown,...,2025-02-24 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO


In [40]:
find_member_booking = df_analytic.copy()

find_member_booking.loc[find_member_booking['TelNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['MobileNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'
find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

find_member_booking.replace('empty', '', inplace=True)

In [41]:
find_member_booking.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\cek_member_zoho_analytic.csv', index=False)

In [42]:
count_dedup_member = df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])].shape[0]


df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])]


,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
14,KK068857,Karma Resorts,Other,Past Guests,Karma Kandara,PF301781,Ms,Kim,Saunders,Unknown,...,2025-03-06 00:00:00,6,DIGITAL,GHPA,DIGITAL,3,0,0,Email and Phone,NO
31,KB046036,Karma Resorts,Other,Past Guests,Karma Bavaria,PF065347,Frau,Marion,Peckmann,Female,...,2025-02-15 00:00:00,1,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
32,KB046215,Karma Resorts,Other,Past Guests,Karma Bavaria,PF213986,Frau,Sabine,Reiner,Female,...,2025-02-16 00:00:00,2,COMP,GER_OWE,COMP,3,0,0,Email and Phone,NO
42,KB046319,Karma Resorts,Other,Past Guests,Karma Bavaria,PF216954,Mr,David,Raguin,Male,...,2025-02-16 00:00:00,2,COMP,GER_OWE,COMP,2,2,0,Email and Phone,NO
48,KB046995,Karma Resorts,Other,Past Guests,Karma Bavaria,PF285336,Frau,Deborah,Hillmann,Female,...,2025-03-02 00:00:00,2,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
50,KB048239,Karma Resorts,Other,Past Guests,Karma Bavaria,PF304831,Frau,Alexandra,Babic,Female,...,2025-02-23 00:00:00,1,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
53,KB047165,Karma Resorts,Other,Past Guests,Karma Bavaria,PF291339,Frau,Karin,Schloßnagl,Female,...,2025-02-09 00:00:00,2,COMP,GER_OWE,COMP,2,1,0,Email Only,NO
54,KB047128,Karma Resorts,Other,Past Guests,Karma Bavaria,PF299070,Frau,Christa,Schwenzer,Female,...,2025-02-09 00:00:00,2,COMP,GER_OWE,COMP,2,0,0,Email Only,NO
62,KB047363,Karma Resorts,Other,Past Guests,Karma Bavaria,PF300795,Herr,Christian,Papenfuhs,Male,...,2025-02-15 00:00:00,6,OTA,OTA,RETAIL,2,1,1,Email and Phone,NO
70,KB046065,Karma Resorts,Other,Past Guests,Karma Bavaria,PF192683,Mrs,Kathrin,Haas,Female,...,2025-03-09 00:00:00,2,COMP,GER_OWE,COMP,2,0,0,Email Only,NO


In [43]:
df_analytic.drop(df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])
].index, inplace=True)

df_analytic.reset_index(drop=True, inplace=True)

count_after_dedup_member = df_analytic.shape[0]

In [44]:
count_data_akhir = df_analytic.shape[0]

In [45]:
df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KSH022009,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF298314,empty,Fiona,Brooker,Unknown,...,2025-02-09 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
1,KB046582,Karma Resorts,Other,Past Guests,Karma Bavaria,PF297461,Herr,Michael,Häusler,Male,...,2025-02-09 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
2,KLM007677,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF297011,Miss,Lynda,Marr,Female,...,2025-02-09 00:00:00,1,DIR,EXT,KC,2,0,0,Email and Phone,NO
3,KK062375,Karma Resorts,Other,Past Guests,Karma Kandara,PF289493,Mr,Son,Youngmi,Male,...,2025-02-09 00:00:00,4,TA,TA,RETAIL,6,0,0,Email and Phone,NO
4,KB047705,Karma Resorts,Other,Past Guests,Karma Bavaria,PF272402,Mrs,Laura,Porter,Female,...,2025-02-09 00:00:00,2,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,KB047612,Karma Resorts,Other,Past Guests,Karma Bavaria,PF301930,empty,Simone,Rippberger,Unknown,...,2025-03-09 00:00:00,1,OTA,OTA,RETAIL,4,0,0,Email and Phone,NO
155,KK060680,Karma Resorts,Other,Past Guests,Karma Kandara,PF284764,Mr,Teddy,Hermanto,Male,...,2025-03-09 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
156,KK058778,Karma Resorts,Other,Past Guests,Karma Kandara,PF281265,Mr,Adrian,Di Lallo,Male,...,2025-03-09 00:00:00,3,DIGITAL,WEBSITE,DIGITAL,5,2,0,Email and Phone,NO
157,KSH021292,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF293972,empty,Brenda,Wilkinson,Unknown,...,2025-03-10 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO


In [46]:
df_analytic['DOB'].loc[df_analytic['DOB']=='']

Series([], Name: DOB, dtype: string)

In [47]:
df_analytic['DOB'].replace('empty', '', inplace=True)
dob_con_date = pd.to_datetime(df_analytic['DOB'], errors='coerce')
dob_date = dob_con_date.dt.strftime('%Y-%m-%d')

arr_con_date = pd.to_datetime(df_analytic['DateArrival'])
arr_date = arr_con_date.dt.strftime('%Y-%m-%d')

dep_con_date = pd.to_datetime(df_analytic['DateDepart'])
dep_date = dep_con_date.dt.strftime('%Y-%m-%d')

df_analytic['DOB'] = dob_date
df_analytic['DateArrival'] = arr_date
df_analytic['DateDepart'] = dep_date


C:\Users\fajar.fatoni\AppData\Local\Temp\ipykernel_13692\1513714307.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_analytic['DOB'].replace('empty', '', inplace=True)


In [48]:
df_analytic.replace('empty', '', inplace=True)

df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KSH022009,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF298314,,Fiona,Brooker,Unknown,...,2025-02-09,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
1,KLM007676,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF297010,,Lynda,Marr,Unknown,...,2025-02-09,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
2,KK069233,Karma Resorts,Other,Past Guests,Karma Kandara,PF302666,Mr,.,Pavel,Male,...,2025-02-09,7,HOUSE_USE,V_OWNER_NRB,COMP,2,1,0,Email and Phone,NO
3,KB047702,Karma Resorts,Other,Past Guests,Karma Bavaria,PF302828,Frau,Waltraud,Haas,Female,...,2025-02-09,2,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
4,KLM007928,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF299289,Mr,Ian,Brooke,Male,...,2025-02-09,1,DIR,EXT,KC,2,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,KK060680,Karma Resorts,Other,Past Guests,Karma Kandara,PF284764,Mr,Teddy,Hermanto,Male,...,2025-03-09,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
155,KB047612,Karma Resorts,Other,Past Guests,Karma Bavaria,PF301930,,Simone,Rippberger,Unknown,...,2025-03-09,1,OTA,OTA,RETAIL,4,0,0,Email and Phone,NO
156,KK058778,Karma Resorts,Other,Past Guests,Karma Kandara,PF281265,Mr,Adrian,Di Lallo,Male,...,2025-03-09,3,DIGITAL,WEBSITE,DIGITAL,5,2,0,Email and Phone,NO
157,KSH021292,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF293972,,Brenda,Wilkinson,Unknown,...,2025-03-10,3,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO


In [49]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Data Setelah Script Past Guest Sebelumnya : "+str(count_data_awal))
print("Jumlah Data Yang Duplicate : "+str(count_duplicate_general))
print("Jumlah Data Setelah Hapus Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Data Yang Termasuk Member: "+str(count_dedup_member))
print("Jumlah Data Setelah Hapus Member : "+str(count_after_dedup_member))
print("Jumlah Data Akhir : "+str(count_data_akhir))

Summary
Jumlah Data Setelah Script Past Guest Sebelumnya : 269
Jumlah Data Yang Duplicate : 86
Jumlah Data Setelah Hapus Duplicate : 183
Jumlah Data Yang Termasuk Member: 24
Jumlah Data Setelah Hapus Member : 159
Jumlah Data Akhir : 159


In [50]:
df_analytic.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\final_analytics.xlsx', index=False)